In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.reset_defaults
# sns.set_style(style='darkgrid')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'size'   : 16}
plt.rc('font', **font)
plt.ion()
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams['figure.figsize'] = (20.0, 10.0)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
cd ..

/Users/jeremymiller/GoogleDrive/Data_Science/Galvanize/dsi-capstone


In [3]:
import src.features.build_features as bf

In [4]:
cd src/

/Users/jeremymiller/GoogleDrive/Data_Science/Galvanize/dsi-capstone/src


## Features in student info

In [ ]:
df = pd.read_csv('../data/raw/studentInfo.csv')

In [ ]:
df.info()

In [ ]:
df['final_result'].unique()

In [ ]:
df = bf.drop_nulls(df)

In [ ]:
cols = ['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability']

In [ ]:
df = bf.one_hot(df, cols)

In [ ]:
df_trans = bf._encode_target(df)

In [ ]:
# df_trans.head()

In [ ]:
model = RandomForestClassifier()

In [ ]:
X = df_trans.drop(['id_student', 'final_result', 'module_completed',
                   'code_module', 'code_presentation'], axis = 1)

In [ ]:
y = df_trans['module_completed']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

## Create features from assessments

In [ ]:
# join assessments to student assessments

In [11]:
std_asmt_df = pd.read_csv('../data/raw/studentAssessment.csv')

In [12]:
asmt_df = pd.read_csv('../data/raw/assessments.csv')

In [13]:
joint_ass = bf._join_asssessments(std_asmt_df, asmt_df)

In [14]:
joint_ass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170874 entries, 0 to 173911
Data columns (total 11 columns):
id_assessment           170874 non-null int64
id_student              170874 non-null int64
date_submitted          170874 non-null float64
is_banked               170874 non-null float64
score                   170874 non-null float64
code_module             170874 non-null object
code_presentation       170874 non-null object
assessment_type         170874 non-null object
date                    170874 non-null float64
weight                  170874 non-null float64
days_submitted_early    170874 non-null float64
dtypes: float64(6), int64(2), object(3)
memory usage: 15.6+ MB


In [15]:
ass_features = bf._averages_from_assessments(joint_ass)

In [16]:
joint_ass.groupby(['id_student'])

In [17]:
ass_features.head()

avg_score  avg_days_sub_early  \
code_module code_presentation id_student                                  
AAA         2014J             6516        61.800000            2.600000   
DDD         2013J             8462        87.666667            0.333333   
            2014J             8462        86.500000           59.500000   
AAA         2013J             11391       82.000000            1.800000   
BBB         2013B             23629       82.500000           -3.500000   

                                          est_final_score  
code_module code_presentation id_student                   
AAA         2014J             6516                  63.50  
DDD         2013J             8462                  34.90  
            2014J             8462                  43.00  
AAA         2013J             11391                 82.40  
BBB         2013B             23629                 16.69

In [ ]:
a.head()

In [ ]:
a['id_student'] = a['id_student'].astype('int64')

In [ ]:
a.rename({'date':'due_date'}, axis = 'columns', inplace=True)

In [ ]:
df3 = a

In [ ]:
df3.head()

In [ ]:
df3['days_submitted_early'] = df3['due_date'] - df3['date_submitted']

In [ ]:
df3.head()

In [ ]:
# get final grade?
# create new column: weighted score
df3['weighted_score'] = df3['score'] * df3['weight'] / 100


In [ ]:
f_df = df3.groupby(by=['id_student', 'code_module', 'code_presentation']).sum()[['weighted_score']]

In [ ]:
f_ass = df3.groupby(by=['id_student', 'code_module', 'code_presentation']).mean()[['score', 'days_submitted_early']]

In [ ]:
all_ass = pd.merge(f_ass, f_df, how='outer', on = ['code_module', 'code_presentation',
                                         'id_student'])

In [ ]:
all_ass[all_ass['weighted_score']>100].count()

In [ ]:
all_ass.info()

In [ ]:
# pd.merge(df, f_ass, how='outer', on = ['code_module', 'code_presentation', 'id_student'])

### VLE

In [26]:
st_vle_df = pd.read_csv('../data/raw/studentVle.csv')
vle_df = pd.read_csv('../data/raw/vle.csv')

In [27]:
joined_vle = bf._join_vle(st_vle_df, vle_df)

In [28]:
joined_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type
0,AAA,2013J,28400.0,546652,-10.0,4.0,forumng
1,AAA,2013J,28400.0,546652,-10.0,1.0,forumng
2,AAA,2013J,28400.0,546652,-10.0,1.0,forumng
3,AAA,2013J,28400.0,546652,-10.0,8.0,forumng
4,AAA,2013J,30268.0,546652,-10.0,3.0,forumng


In [ ]:
# features: total number of clicks, median date, # of days accessed, date with most clicks, (per activity type)

## Registrations

In [ ]:
r_df = pd.read_csv('../data/raw/studentRegistrations.csv')
s_df = pd.read_csv('../data/raw/studentInfo.csv')

In [ ]:
r = pd.merge(s_df, r_df, how='outer', on=['code_module', 'code_presentation',
                                      'id_student']).fillna(value=0)

In [ ]:
r.isnull().sum()